In [1]:
import pandas as pd
import re

#Funzione di supporto

In [2]:
def pulisci_titolo(titolo):
    # Rimuove l'indice numerico iniziale, es. "1. " → ""
    titolo = re.sub(r'^\d+\.\s*', '', titolo)
    
    # Rimuove articoli iniziali (The, A, An) solo se sono all'inizio
    titolo = re.sub(r'^(The|A|An)\s+', '', titolo, flags=re.IGNORECASE)
    
    return titolo

def durata_in_minuti(durata_str):
    if isinstance(durata_str, str):
        match = re.match(r'(?:(\d+)h)?\s*(?:(\d+)m)?', durata_str.strip())
        ore = int(match.group(1)) if match and match.group(1) else 0
        minuti = int(match.group(2)) if match and match.group(2) else 0
        return ore * 60 + minuti
    else:
        return 0  # oppure puoi restituire None

# Caricamento del file CSV in un dataframe

In [3]:
# Carica il file CSV in un DataFrame
df = pd.read_csv("imdb_movies_2024.csv")

#1. Ordinamento dei film in base al titolo

In [7]:
# Dataframe con titoli ordinati
df['Title_senza_articoli'] = df['Title'].apply(pulisci_titolo)

# Ordina in base alla colonna pulita, poi rimuovila
df_ordinato = (
    df.sort_values(by='Title_senza_articoli')
      .drop(columns=['Title_senza_articoli'])
      .reset_index(drop=True)
)

print(df_ordinato)

                                                 Title  \
0                                414. A '90s Christmas   
1    114. 2024 Oscar Nominated Short Films: Documen...   
2                                            305. 2073   
3                                  524. The 4:30 Movie   
4                                         316. 50 km/h   
..                                                 ...   
595                                  325. Young Hearts   
596                       194. Young Woman and the Sea   
597                                  174. Your Monster   
598                              291. Zurawski v Texas   
599                         229. ¡Casa Bonita Mi Amor!   

                                            Movie Link  Year Duration  MPA  \
0    https://www.imdb.com/title/tt16270280/?ref_=sr...  2024   1h 30m  NaN   
1    https://www.imdb.com/title/tt31272312/?ref_=sr...  2024   2h 13m  NaN   
2    https://www.imdb.com/title/tt22042346/?ref_=sr...  2024   1h 25m

#2. Filtraggio dei film con durata > 120 minuti

In [8]:
df['Durata_in_minuti'] = df['Duration'].apply(durata_in_minuti)

"""
df_sorted_by_durata = df[df['Durata_in_minuti'] >= 120].reset_index(drop=True)  #maschera booleana
"""

df_120_minutes_movies = (df.query("Durata_in_minuti >= 120")
.sort_values(by="Durata_in_minuti")
.reset_index(drop=True)
)

print(df_120_minutes_movies)

                                          Title  \
0                                  46. Conclave   
1                        351. How to Make Gravy   
2                                     340. Eden   
3                               329. Culpa Tuya   
4                                  300. Santosh   
..                                          ...   
153                           66. Kalki 2898 AD   
154   49. Horizon: An American Saga - Chapter 1   
155  377. Horizon: An American Saga - Chapter 2   
156               76. Pushpa: The Rule - Part 2   
157                          167. The Brutalist   

                                            Movie Link  Year Duration  \
0    https://www.imdb.com/title/tt20215234/?ref_=sr...  2024       2h   
1    https://www.imdb.com/title/tt26315685/?ref_=sr...  2024       2h   
2    https://www.imdb.com/title/tt23149780/?ref_=sr...  2024       2h   
3    https://www.imdb.com/title/tt28510079/?ref_=sr...  2024       2h   
4    https://www.imdb.

#3. Raggruppamento per MPA

In [9]:
conteggio_mpa = df.groupby('MPA').size().reset_index(name='Totale_film')
print(conteggio_mpa)

          MPA  Totale_film
0         13+            1
1         16+            1
2           G            1
3   Not Rated           15
4          PG           37
5       PG-13           82
6           R          184
7       TV-14           11
8        TV-G            1
9       TV-MA           23
10      TV-PG            9
11       TV-Y            1
12    Unrated            3


#4. Filtraggio dei film con rating > 7.5

In [10]:
df_rating_alto = (df.query("Rating > 7.5")
.sort_values(by="Rating", ascending=False)
.reset_index(drop=True))

print(df_rating_alto[['Title', 'Rating', 'Durata_in_minuti']])

                                                Title  Rating  \
0                                  241. Water Brother     9.5   
1   170. Laufey's A Night at the Symphony: Hollywo...     9.4   
2     468. Attack on Titan the Movie: The Last Attack     9.3   
3                                  581. Desert Fiends     9.2   
4                            277. Inheritance to Love     9.2   
..                                                ...     ...   
89                                   208. Nickel Boys     7.6   
90                          221. Bocchi the Rock! Re:     7.6   
91                     244. The Count of Monte-Cristo     7.6   
92                          542. Love in the Big City     7.6   
93                                490. The Bibi Files     7.6   

    Durata_in_minuti  
0                  0  
1                 98  
2                145  
3                  0  
4                 88  
..               ...  
89               140  
90                90  
91          